In [1]:
!git clone https://github.com/raghav-thiruv/Omdena-Quantum-Self-Driving

Cloning into 'Omdena-Quantum-Self-Driving'...
remote: Enumerating objects: 45896, done.
remote: Counting objects: 100% (11806/11806), done.
remote: Compressing objects: 100% (11707/11707), done.
remote: Total 45896 (delta 98), reused 11794 (delta 89), pack-reused 34090
Receiving objects: 100% (45896/45896), 2.16 GiB | 37.67 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Updating files: 100% (45598/45598), done.


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import pandas as pd
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# Load steering angles from text file
df = pd.read_csv('/kaggle/working/Omdena-Quantum-Self-Driving/Images/driving_dataset1/data.txt', names=['filename', 'steering_angle'], delimiter=' ')
image_dir = '/kaggle/working/Omdena-Quantum-Self-Driving/Images/driving_dataset1/'
df['filename'] = df['filename'].apply(lambda x: os.path.join(image_dir, x))
df.head()

,filename,steering_angle
0,/kaggle/working/Omdena-Quantum-Self-Driving/Im...,0.0
1,/kaggle/working/Omdena-Quantum-Self-Driving/Im...,0.0
2,/kaggle/working/Omdena-Quantum-Self-Driving/Im...,0.0
3,/kaggle/working/Omdena-Quantum-Self-Driving/Im...,0.0
4,/kaggle/working/Omdena-Quantum-Self-Driving/Im...,0.0


In [4]:
# Load images from the paths
def load_image(image_path):
    try:
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (224, 224))
        image = tf.keras.applications.mobilenet.preprocess_input(image)
    except:
        print(f"Invalid image format, skipping: {image_path}")
        return None
    return image

# Create dataset from images
def create_dataset(df):
    image_dataset = tf.data.Dataset.from_tensor_slices(df['filename'])
    angle_dataset = tf.data.Dataset.from_tensor_slices(df['steering_angle'])
    image_dataset = image_dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
    image_dataset = image_dataset.apply(tf.data.experimental.ignore_errors())
    dataset = tf.data.Dataset.zip((image_dataset, angle_dataset))

    return dataset

In [5]:
# Mobilenet architecture
mobilenet= tf.keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet', input_shape=(224,224,3))
x= mobilenet.output
x= tf.keras.layers.MaxPooling2D()(x)
x= tf.keras.layers.Flatten()(x)
x= tf.keras.layers.Dense(4096, activation= 'relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
output= tf.keras.layers.Dense(1)(x)
model= tf.keras.Model(inputs= mobilenet.input, outputs= output)

    
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mean_absolute_error') 


17225924/17225924 [==============================] - 0s 0us/step


In [6]:
# Split for training and validation
validation_split = 0.1
df = df.sample(frac=1).reset_index(drop=True)
val_df = df[:int(validation_split*len(df))]
train_df = df[int(validation_split*len(df)):]

train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)

batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [7]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [8]:
# Fit the model
# add accuracy metrics tomorrow
model.fit(train_dataset, validation_data=val_dataset, epochs=25, callbacks=[early_stopping])

Epoch 1/25
1278/1278 [==============================] - 197s 127ms/step - loss: 9.6574 - val_loss: 7.1423
Epoch 2/25
1278/1278 [==============================] - 165s 127ms/step - loss: 6.7051 - val_loss: 5.9077
Epoch 3/25
1278/1278 [==============================] - 165s 127ms/step - loss: 5.4660 - val_loss: 4.7929
Epoch 4/25
1278/1278 [==============================] - 165s 128ms/step - loss: 4.6524 - val_loss: 4.1283
Epoch 5/25
1278/1278 [==============================] - 166s 129ms/step - loss: 3.9372 - val_loss: 3.5377
Epoch 6/25
1278/1278 [==============================] - 164s 127ms/step - loss: 3.2920 - val_loss: 2.7861
Epoch 7/25
1278/1278 [==============================] - 164s 127ms/step - loss: 2.7430 - val_loss: 2.4104
Epoch 8/25
1278/1278 [==============================] - 164s 127ms/step - loss: 2.3783 - val_loss: 2.2277
Epoch 9/25
1278/1278 [==============================] - 166s 128ms/step - loss: 2.1476 - val_loss: 2.0679
Epoch 10/25
1278/1278 [=======================

In [9]:
#Testing

In [13]:
import random
from tensorflow.keras.preprocessing import image as keras_image
import numpy as np

def load_image(image_path, target_size=(224, 224)):
    img = keras_image.load_img(image_path, target_size=target_size)
    img_tensor = keras_image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor = tf.keras.applications.mobilenet.preprocess_input(img_tensor)
    return img_tensor

for i in range(1,11):
    
    print("Test case:", i)
    # Randomly select an image from validation set
    random_image_path = random.choice(val_df['filename'].tolist())

    # Load the image
    test_image = load_image(random_image_path)

    # Use the model to predict the steering angle for the test image
    predicted_angle = model.predict(test_image)

    # Print out the predicted steering angle
    print("Predicted steering angle: ", predicted_angle[0][0])

    # If you want to compare this prediction to the actual angle, you could find that as follows:
    actual_angle = val_df[val_df['filename'] == random_image_path]['steering_angle'].values[0]
    print("Actual steering angle: ", actual_angle)


Test case: 1
1/1 [==============================] - 0s 24ms/step
Predicted steering angle:  7.6392317
Actual steering angle:  8.27
Test case: 2
1/1 [==============================] - 0s 23ms/step
Predicted steering angle:  -6.1981015
Actual steering angle:  -8.87
Test case: 3
1/1 [==============================] - 0s 21ms/step
Predicted steering angle:  -9.359783
Actual steering angle:  -10.18
Test case: 4
1/1 [==============================] - 0s 21ms/step
Predicted steering angle:  -4.345709
Actual steering angle:  -5.85
Test case: 5
1/1 [==============================] - 0s 21ms/step
Predicted steering angle:  13.128792
Actual steering angle:  9.98
Test case: 6
1/1 [==============================] - 0s 22ms/step
Predicted steering angle:  0.41522974
Actual steering angle:  0.91
Test case: 7
1/1 [==============================] - 0s 23ms/step
Predicted steering angle:  -30.569344
Actual steering angle:  -30.86
Test case: 8
1/1 [==============================] - 0s 23ms/step
Predicted

In [14]:
model.save('mobilenetv3.h5')